## Import libraries

In [ ]:
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor

import matplotlib.pyplot as plt

## Fetch data

In [ ]:
stock_symbol = 'GOOGL'
start_date = '2020-01-01'
end_date = '2023-01-01'

# Fetch stock data using yfinance
stock_data = yf.download(stock_symbol, start=start_date, end=end_date, progress=False)
stock_data

In [ ]:
stock_data.tail()

## Preprocess data

In [ ]:
# Use 'Close' prices for prediction
data = stock_data['Close'].values.reshape(-1, 1)

# Normalize the data to values between 0 and 1
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Create time series train and test sets

In [ ]:
lookback = 30  # Number of previous days to consider
X, y = [], []
for i in range(len(data) - lookback):
    X.append(data[i:i+lookback, 0])
    y.append(data[i+lookback, 0])

X, y = np.array(X), np.array(y)
train_size = int(0.8 * len(X))

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

## Build and train model

In [ ]:
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

## Predictions

In [ ]:
y_pred_xgb = xgb_model.predict(X_test.reshape(X_test.shape[0], -1))
y_pred_xgb = scaler.inverse_transform(y_pred_xgb.reshape(-1, 1))
y_orig = scaler.inverse_transform(y_test.reshape(-1, 1))

## Evaluate model

In [ ]:
mse_xgb = mean_squared_error(y_orig, y_pred_xgb)
mse_xgb

In [ ]:
test_dates = stock_data.index[train_size + lookback:].to_list()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(test_dates, y_orig, label='Actual', color='black')
plt.plot(test_dates, y_pred_xgb, label='XGBoost Predicted', color='red')
plt.legend()
plt.title(f'Predictions vs Actual\nXGBoost MSE: {mse_xgb:.4f}')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.xticks(rotation=45)
plt.show()